In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%time data = pd.read_csv('../train.csv')

CPU times: user 20.2 s, sys: 1.48 s, total: 21.6 s
Wall time: 24.3 s


In [3]:
test = pd.read_csv('../test.csv')

In [4]:
samplesub = pd.read_csv('../sample_submission.csv')

In [31]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import unicodedata

[nltk_data] Downloading package stopwords to
[nltk_data]     /store/tveiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
norm = lambda w : unicodedata.normalize('NFKD', w).encode('ASCII', 'ignore').decode('ASCII')
all_stopw = set()
for corpus in ['english', 'portuguese', 'spanish']:
    all_stopw.update(set(map(norm, stopwords.words(corpus))))

NameError: name 'unicodedata' is not defined

<hr>

In [5]:
data = data.sample(frac = 1) # keep index!

In [6]:
def normalize(curr):
    # remove accent
    curr = curr.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    # to lower case
    curr = curr.str.lower()
    # remove not alphanumerics or . ,
    curr = curr.str.replace('[^a-zA-Z0-9.,]', ' ')
    
    # let , and . be the same char
    curr = curr.str.replace('[.]', ',')
    
    # remove . , not between numbers
    curr = curr.str.replace('(?<=[0-9])[,]+(?=[0-9])', '.')
    curr = curr.str.replace('[,]', ' ')
    
    # set all digits to 0
    curr = curr.str.replace('[0-9]', '0')
    
    # separate ' <digits><letters ' like in 22g or 12ms
    # curr = curr.str.replace('(^| )([0-9]+)([a-zA-Z]+)($| )', r'\1\2 \3\4')
    
    # remove some Pt plurals
    curr = curr.str.replace('\\b([a-zA-Z]+[aeiouwy])(s)\\b', r'\1')
    
    # remove 4 consec (same) letters to just one
    curr = curr.str.replace(r'([a-zA-Z])\1{3,}', r'\1') # 3 is four? -> three of \1 after first \1...
    
    # separate 4 or more consecutive (different or not) letters
    curr = curr.str.replace(r'([a-zA-Z]{4,})', r' \1 ')
    
    # Other ideas: 
    
    return curr

In [7]:
X_data = data.title
test_data = test.title

In [8]:
X_full = pd.concat([X_data, test_data])

In [9]:
%%time 
X_full = normalize(X_full)

CPU times: user 5min 22s, sys: 3.05 s, total: 5min 25s
Wall time: 5min 18s


In [10]:
sp2 = len(data)

In [13]:
%%time
xremo = lambda s : ' '.join([w for w in s if w not in all_stopw])
X_full = X_full.str.split().apply(xremo)

CPU times: user 39.4 s, sys: 2.61 s, total: 42 s
Wall time: 41.4 s


In [16]:
%%time
# remove tokens with numbers -> is it necessary at this level?
# X_full = X_full.str.replace('[a-z0-9_]*[0-9][a-z0-9_]*', ' ')

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 14.1 µs


In [18]:
train_norm = X_full[:sp2]
test_norm = X_full[sp2:]

In [19]:
%%time 
# train_words = X_norm.str.split(expand=True).stack().value_counts().to_dict()
train_words = dict()
for t in train_norm:
    for w in t.split():
        train_words[w] = train_words.get(w, 0) + 1

CPU times: user 38.4 s, sys: 7.86 ms, total: 38.4 s
Wall time: 38.4 s


In [20]:
%%time 
# test_words = test_norm.str.split(expand=True).stack().value_counts().to_dict()
test_words = dict()
for t in test_norm:
    for w in t.split():
        test_words[w] = test_words.get(w, 0) + 1

CPU times: user 442 ms, sys: 3.91 ms, total: 446 ms
Wall time: 445 ms


In [21]:
len(test_words), len(train_words)

(79429, 915881)

In [22]:
testwseries = pd.Series(test_words).index

In [23]:
temp_test = testwseries.isin(pd.Series(train_words).index)

In [24]:
temp_test.mean()

0.9490991955079379

In [25]:
testwseries[~temp_test].values[-200:]

array(['lacrex', 'biomagneticasx', 'dynamicut', 'drtagon', 'noboru',
       'capacipativa', 'cardiocom', 'protrectore', 'pl00sf000a',
       'lc00exn', 'duplivox', 'microdin', 'plotteable', 'mgr0000',
       'aeroburne', 'megafome', 'pecorelli', 'abdefg', 'upffactor',
       'armatazo', 'rockernapper', 'cormans', 'swereco', 'hirvonen',
       'propicepci', 'pitosporium', 'lidschatten', 'c0mtc', 'tocadidco',
       'conns', 'danning', 'dp000g', 'caseprotetora', 'premiumkw', 'hy0a',
       'dabin', 'littly', 'sousplatss', 'giracar', 'an0r', 'adpitadop',
       'radifon', 'antibalistica', 'dslnikon', 'servicegy', 'megaxone',
       'pasadico', 'tagforce', 'sellegra', 'natuel', 'neckknife',
       'stillesistema', 'aoske', 'zoadiaco', 'canixhito', 'oktoberfe',
       '000wvp', 'dallacost', 'atraum', 'compesa', 'sldbl', 'rindete',
       'thiriet', 'fornotedesco', 'gasthermoval', 'microconvexa', 'ssggg',
       'megadone', 'mayoristaozanamx', 'adesg', 'zapemdia', 'melloding',
       'bototn

In [26]:
%%time
## how many instances are necessary (assuming X_norm is shuffled already)
test_counter = test_words.copy()
priorities = np.full(len(X_norm), 999_999)

for i, t in enumerate(X_norm):
    for w in t.split():
        if w in test_counter:
            priorities[i] = min(priorities[i], test_words[w] - test_counter[w])
            test_counter[w] -= 1 # 1 for equal ammount, .5 for twice in train, ... # actually any positive val works


CPU times: user 1min 58s, sys: 27.8 ms, total: 1min 58s
Wall time: 1min 58s


In [27]:
data['priorities'] = priorities

In [28]:
data.sort_values(['category', 'label_quality', 'priorities'], inplace=True)

In [29]:
%time reduced = data.groupby(['category'], as_index=False).head(800)

CPU times: user 1.15 s, sys: 332 ms, total: 1.48 s
Wall time: 1.48 s


In [32]:
%%time 
red_norm = normalize(reduced.title)
# red_norm = red_norm.str.replace('[a-z0-9_]*[0-9][a-z0-9_]*', ' ')
red_norm = red_norm.str.split().apply(xremo)

CPU times: user 19.9 s, sys: 0 ns, total: 19.9 s
Wall time: 19.9 s


In [33]:
%%time 
red_words = red_norm.str.split(expand=True).stack().value_counts().to_dict()

CPU times: user 10.5 s, sys: 401 ms, total: 10.9 s
Wall time: 9.68 s


In [34]:
temp2_test = testwseries.isin(pd.Series(red_words).index)

In [38]:
temp2_test.mean(), temp2_test.sum() # 800 ->937, 1000 -> 94

(0.9286406728021251, 73761)

In [36]:
%%time 
left = data.loc[~data.index.isin(reduced.index)].query('priorities <= 5') # 0 -> 1650, 1,0 -> 32k, 1-5 -> 97k, 1-10 -> 188k

CPU times: user 9.01 s, sys: 608 ms, total: 9.61 s
Wall time: 6.19 s


In [37]:
len(left)

119938

In [39]:
reduced = pd.concat([reduced, left]) # next time do a shuffle after this

In [40]:
reduced = reduced.sample(frac = 1)

In [41]:
%%time 
red_norm = normalize(reduced.title)
# red_norm = red_norm.str.replace('[a-z0-9_]*[0-9][a-z0-9_]*', ' ')
red_norm = red_norm.str.split().apply(xremo)
red_words = red_norm.str.split(expand=True).stack().value_counts().to_dict()
temp2_test = testwseries.isin(pd.Series(red_words).index)

CPU times: user 22.7 s, sys: 11.9 ms, total: 22.7 s
Wall time: 22.7 s


In [45]:
temp2_test.mean()

0.9490991955079379

In [46]:
(~temp2_test).sum()

4043

In [47]:
len(red_norm)

1375823

<hr>

## Data Simplification

In [48]:
from sklearn.preprocessing import LabelEncoder

In [49]:
le = LabelEncoder()

In [50]:
le.fit(data.category)

LabelEncoder()

In [51]:
# must save this for later

le.classes_

array(['3D_GLASSES', '3D_PENS', '3D_PRINTERS', ..., 'YARNS', 'YOGA_MATS',
       'YOGURT_MAKERS'], dtype=object)

In [52]:
# change target to numeric

reduced['category'] = le.transform(reduced.category)

In [53]:
# change label_quality to binary integer

reduced['label_quality'] =  reduced.label_quality.map({'reliable' : 0, 'unreliable' : 1})

In [54]:
# change language to binary integer

reduced['language'] =  reduced.language.map({'spanish' : 0, 'portuguese' : 1})

In [55]:
reduced.head()

,title,label_quality,language,category,priorities
473424,Kit 04 Jogo De Lençol De Berço Em Malha 3 Pcs ...,0,1,114,6245
7519083,Bomba Submersa 450 W Agua Suja Turva Limpa Bsv...,1,1,1360,4
19488607,Nadador Tiburon Ys1378-5,1,0,1155,54
16895633,Máscara Angry Birds 6un Imbatível,0,1,1102,486
10369454,Aparador Fruteira Madeira De Demolição 1 Gavet...,0,1,1288,1075


In [56]:
reduced.shape

(1375823, 5)

In [57]:
reduced.to_csv('../data-reduced-800-v3-shuffled.csv', index = True)